## Baixa conteudo da web e salva em disco

In [5]:
## Baixa conteudo da web e salva em disco
import requests
import os

def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def get_file_name(url:str, response:requests.Response) -> str:
        file_name = 'download'
        if 'Content-Disposition' in response.headers:
            content_disposition = response.headers.get('Content-Disposition')
            file_name = content_disposition.split('filename=')[1].strip('"')
        else:
            file_name = url.split("/")[-1]
        return file_name

def generate_available_filename(directory, filename):

    name, extension = os.path.splitext(filename)
    file_path = os.path.join(directory, filename)
    counter = 1

    while os.path.exists(file_path):
        new_filename = f"{name} ({counter}){extension}"
        file_path = os.path.join(directory, new_filename)
        counter += 1

    return file_path

def download(url:str, directory:str=os.getcwd()) -> str:
    try:
        response = requests.get(url)
        response.raise_for_status()
        file_name = generate_available_filename(directory, get_file_name(url, response))
        create_directory_if_not_exists(directory)
        with open(generate_available_filename(directory, get_file_name(url, response)), "wb") as file:
            file.write(response.content)

        return file_name

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        raise e

## File Utils

In [6]:
import os
import zipfile

def unzip(zip_path, dataset_unzip_directory, file_to_extract) -> str:
     remove_file(os.path.join(dataset_unzip_directory, file_to_extract))
     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
         return zip_ref.extract(file_to_extract, dataset_unzip_directory)
 
 
def remove_file(caminho_arquivo):
     if os.path.exists(caminho_arquivo):
         os.remove(caminho_arquivo)
     else:
         print(f"Arquivo '{caminho_arquivo}' não encontrado.")   

def move_file(origem, destino):
     os.replace(origem, destino)

def remove_directory(directory):
     if os.path.exists(directory):
         os.rmdir(directory)

## CSV Utils

In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import dask.dataframe as dd

def convert_csv_to_parquet(csv_path):
    """Função para converter CSV em Parquet."""
    try:
        # csv_path = convert_csv_encoding(csv_path, 'latin1', 'utf-8')
        df = pd.read_csv(csv_path, sep=';', encoding='latin1', low_memory=False)
        parquet_path = csv_path.replace(".csv", ".parquet")
        df.to_parquet(parquet_path, index=False)
        return parquet_path
    except Exception as e:
        print(f"Erro ao converter {csv_path} para Parquet: {e}")
        return None

def convert_csv_encoding(csv_path, source_encoding, target_encoding='utf-8') -> str:
    """Função para converter a codificação de um arquivo CSV."""
    try:
        df = dd.read_csv(csv_path, encoding=source_encoding)
        remove_file(csv_path)
        df.to_csv(csv_path, sep=';', encoding=target_encoding, index=False)
        return csv_path
    except Exception as e:
        print(f"Erro ao converter a codificação do arquivo {csv_path}: {e}")
        raise e

def convert_parquet_encoding(input_parquet_path, output_parquet_path, source_encoding='latin1', target_encoding='utf-8'):
    df = pd.read_parquet(input_parquet_path)
    
    for col in df.select_dtypes(include=[object]):
        df[col] = df[col].apply(lambda x: x.encode(source_encoding).decode(target_encoding) if isinstance(x, str) else x)
    
    table = pa.Table.from_pandas(df)
    pq.write_table(table, output_parquet_path)
    
    print(f"Arquivo Parquet convertido de Latin1 para UTF-8 e salvo em {output_parquet_path}")


## Prepara datasets para serem ingeridos pelo notebook

In [ ]:
import os

resources_directory = os.path.join(os.getcwd(), "resources")

dataset_directory = os.path.join(os.getcwd(), "dataset/2022")
dataset_unzip_directory= os.path.join(dataset_directory, "unzip")


## DataSet Principal (Candidatos)
resource_file = os.path.join(resources_directory, "consulta_cand_2022.zip")
if not os.path.exists(resource_file):
    URL = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip'
    resource_file = download(URL, resources_directory)

file_to_extract = "consulta_cand_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    # convert_parquet_encoding(parquet_file, os.path.join(dataset_directory, "candidatos.parquet"))
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.parquet"))

## DataSet Complementar (Informacoes Complementares)
resource_file = os.path.join(resources_directory, "consulta_cand_complementar_2022.zip")
if not os.path.exists(resource_file):
    URL = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand_complementar/consulta_cand_complementar_2022.zip'
    resource_file = download(URL, resources_directory)

file_to_extract = "consulta_cand_complementar_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.infos_adicionais.parquet"))


## DataSet Bens dos Candidatos
resource_file = os.path.join(resources_directory, "bem_candidato_2022.zip")
if not os.path.exists(resource_file):
    URL = 'https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2022.zip'
    resource_file = download(URL, resources_directory)

file_to_extract = "bem_candidato_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.bens.parquet"))



## DataSet Prestação de Contas
resource_file = os.path.join(resources_directory, "prestacao_de_contas_eleitorais_candidatos_2022.zip")
if not os.path.exists(resource_file):
    URL = 'https://cdn.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_de_contas_eleitorais_candidatos_2022.zip'
    resource_file = download(URL, resources_directory)


## Despesas Contratadas
file_to_extract = "despesas_contratadas_candidatos_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.despesas.contratadas.parquet"))

## Despesas Contratadas Pagas
file_to_extract = "despesas_pagas_candidatos_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.despesas.pagas.parquet"))

## Receitas
file_to_extract = "receitas_candidatos_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.receitas.parquet"))

## Receitas
file_to_extract = "receitas_candidatos_doador_originario_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "candidatos.receitas.doador_originario.parquet"))

remove_directory(dataset_unzip_directory)

In [ ]:
import os

resources_directory = os.path.join(os.getcwd(), "resources")

dataset_directory = os.path.join(os.getcwd(), "dataset/2022")
dataset_unzip_directory= os.path.join(dataset_directory, "unzip")


## DataSet Resultados
resource_file = os.path.join(resources_directory, "votacao_candidato_munzona_2022.zip")
if not os.path.exists(resource_file):
    URL = 'https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2022.zip'
    resource_file = download(URL, resources_directory)

file_to_extract = "votacao_candidato_munzona_2022_BRASIL.csv"
file_extracted = unzip(resource_file, dataset_unzip_directory, file_to_extract)
parquet_file = convert_csv_to_parquet(file_extracted)
if parquet_file:
    remove_file(file_extracted)
    move_file(parquet_file, os.path.join(dataset_directory, "resultados.votacao.canditados.parquet"))

remove_directory(dataset_unzip_directory)

# Preparação dos dados

In [19]:
from typing import List

import duckdb


class ParquetView:
    def __init__(self, parquet_file:str, view_name:str):
        self.parquet_file = parquet_file
        self.view_name = view_name

    def connection_string(self):
        return f"CREATE VIEW '{self.view_name}' AS SELECT * FROM '{self.parquet_file}'"
    
def create_view(parquet_views:List[ParquetView], con=duckdb.connect(database=':memory:')) -> duckdb.DuckDBPyConnection:
    for view in parquet_views:
        con.execute(view.connection_string())
    return con

def materialize_query_into_parquet(query:str, parquet_file:str, con:duckdb.DuckDBPyConnection) -> duckdb.DuckDBPyConnection:
    con.execute(query).fetchdf().to_parquet(parquet_file, index=False)
    return con

In [62]:
### EXTRAI TABELAS CONTIDAS EM candidatos.parquet

import os

dataset_directory = os.path.join(os.getcwd(), "dataset", "2022")
tables_directory = os.path.join("dataset", "2022", "candidatos", "tabelas")
os.makedirs(tables_directory, exist_ok=True)

con = duckdb.connect()
candidatos_view = ParquetView(os.path.join(dataset_directory, "candidatos.parquet"), "candidatos")
create_view([candidatos_view], con)


## TIPOS_ELEICAO
query = """
   SELECT distinct
        CD_TIPO_ELEICAO as ID,
        NM_TIPO_ELEICAO as TIPO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "tipos_eleicao.parquet"), con)


#ELEICOES
query = """
   SELECT distinct
        CD_ELEICAO as ID,
        DS_ELEICAO as DESCRICAO,
        TP_ABRANGENCIA as ABRANGENCIA,
        NR_TURNO as TURNO,
        CD_TIPO_ELEICAO as ID_TIPO_ELEICAO,
        DT_ELEICAO as DATA
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "eleicoes.parquet"), con)

#CARGOS
query = """
   SELECT distinct
        CD_CARGO as ID,
        DS_CARGO as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "cargos.parquet"), con)

#SITUACOES_CANDIDATURA
query = """
   SELECT distinct
        CD_SITUACAO_CANDIDATURA as ID,
        DS_SITUACAO_CANDIDATURA as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "situacoes_candidatura.parquet"), con)

#SITUACOES_CANDIDATURA
query = """
   SELECT distinct
        SG_UF as SIGLA,
        NM_UE as NOME
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "unidades_federativas.parquet"), con)

#FEDERACOES
query = """
   SELECT distinct
        NR_FEDERACAO as ID,
        NM_FEDERACAO as NOME,
        SG_FEDERACAO as SIGLA,
        DS_COMPOSICAO_FEDERACAO as COMPOSICAO
    FROM
        candidatos
    where NR_FEDERACAO != -1
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "federacoes.parquet"), con)


#COLIGACOES
query = """
   SELECT distinct
        SQ_COLIGACAO as ID,
        NM_COLIGACAO as NOME,
        DS_COMPOSICAO_COLIGACAO as COMPOSICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "coligacoes.parquet"), con)

#GENEROS
query = """
   SELECT distinct
        CD_GENERO as ID,
        DS_GENERO as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "generos.parquet"), con)

#GRAUS_INSTRUCAO
query = """
   SELECT distinct
        CD_GRAU_INSTRUCAO as ID,
        DS_GRAU_INSTRUCAO as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "graus_instrucao.parquet"), con)

#ESTADOS_CIVIL
query = """
   SELECT distinct
        CD_ESTADO_CIVIL as ID,
        DS_ESTADO_CIVIL as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "estados_civil.parquet"), con)

#CORES_RACA
query = """
   SELECT distinct
        CD_COR_RACA as ID,
        DS_COR_RACA as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "cores_raca.parquet"), con)

#OCUPACOES
query = """
   SELECT distinct
        CD_OCUPACAO as ID,
        DS_OCUPACAO as DESCRICAO
    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "ocupacoes.parquet"), con)

#SITUACOES_TOTALIZACAO
query = """
   SELECT distinct
        CD_SIT_TOT_TURNO as ID,
        DS_SIT_TOT_TURNO as DESCRICAO
    FROM
        candidatos
    where CD_SIT_TOT_TURNO != -1
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "situacoes_totalizacao.parquet"), con)


#CANDIDATOS
query = """
   SELECT distinct        
        SQ_CANDIDATO as ID,
        CD_ELEICAO as ID_ELEICAO,
        SG_UF as UF,
        NR_PARTIDO as NUMERO_PARTIDO,
        NR_CANDIDATO as NUMERO,
        NM_CANDIDATO as NOME,
        NM_URNA_CANDIDATO as NOME_URNA,
        NR_FEDERACAO as ID_FEDERACAO,
        SQ_COLIGACAO as ID_COLIGACAO,
        SG_UF_NASCIMENTO as UF_NASCIMENTO,
        DT_NASCIMENTO as DATA_NASCIMENTO,
        NR_TITULO_ELEITORAL_CANDIDATO as NUMERO_TITULO_ELEITORAL,
        CD_GENERO as ID_GENERO,
        CD_GRAU_INSTRUCAO as ID_GRAU_INSTRUCAO,
        CD_ESTADO_CIVIL as ID_ESTADO_CIVIL,
        CD_COR_RACA as ID_COR_RACA,
        CD_OCUPACAO as ID_OCUPACAO,
        CD_SIT_TOT_TURNO as ID_SITUACAO_TOTALIZACAO

    FROM
        candidatos
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "candidatos.parquet"), con)


con.close()

In [63]:
### EXTRAI TABELAS CONTIDAS EM candidatos.infos_adicionais.parquet

import os

dataset_directory = os.path.join(os.getcwd(), "dataset", "2022")
tables_directory = os.path.join("dataset", "2022", "candidatos", "tabelas")
os.makedirs(tables_directory, exist_ok=True)

con = duckdb.connect()
candidatos_view = ParquetView(os.path.join(dataset_directory, "candidatos.infos_adicionais.parquet"), "infos_adicionais")
create_view([candidatos_view], con)


## DETALHES_SITUACAO_CANDIDATURA
query = """
   SELECT distinct
        CD_DETALHE_SITUACAO_CAND as ID,
        DS_DETALHE_SITUACAO_CAND as DESCRICAO
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "detalhes_situacao_candidatura.parquet"), con)

## NACIONALIDADES
query = """
   SELECT distinct
        CD_NACIONALIDADE as ID,
        DS_NACIONALIDADE as DESCRICAO
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "nacionalidades.parquet"), con)

## SITUACOES_CANDIDATO_PLEITO
query = """
   SELECT distinct
        CD_SITUACAO_CANDIDATO_PLEITO as ID,
        DS_SITUACAO_CANDIDATO_PLEITO as DESCRICAO
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "situacoes_candidato_pleito.parquet"), con)

## SITUACOES_CANDIDATO_URNA
query = """
   SELECT distinct
        CD_SITUACAO_CANDIDATO_URNA as ID,
        DS_SITUACAO_CANDIDATO_URNA as DESCRICAO
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "situacoes_candidato_urna.parquet"), con)

## SITUACOES_CANDIDATO_TOTALIZACAO
query = """
   SELECT distinct
        CD_SITUACAO_CANDIDATO_TOT as ID,
        DS_SITUACAO_CANDIDATO_TOT as DESCRICAO
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "situacoes_candidato_totalizacao.parquet"), con)

## INFORMACOES_ADICIONAIS
query = """
   SELECT distinct
        SQ_CANDIDATO as ID_CANIDATO,
        CD_DETALHE_SITUACAO_CAND as ID_DETALHE_SITUACAO_CANDIDATURA,
        CD_NACIONALIDADE as ID_NACIONALIDADE,
        NM_MUNICIPIO_NASCIMENTO as NOME_MUNICIPIO_NASCIMENTO,
        NR_IDADE_DATA_POSSE as IDADE_POSSE,
        VR_DESPESA_MAX_CAMPANHA as DESPESA_MAX_CAMPANHA,
        ST_REELEICAO as REELEICAO,
        ST_DECLARAR_BENS as DECLARAR_BENS,
        CD_SITUACAO_CANDIDATO_PLEITO as ID_SITUACAO_CANDIDATO_PLEITO,
        CD_SITUACAO_CANDIDATO_URNA as ID_SITUACAO_CANDIDATO_URNA,
        ST_CANDIDATO_INSERIDO_URNA as INSERIDO_URNA,
        NM_TIPO_DESTINACAO_VOTOS as TIPO_DESTINACAO_VOTOS,
        CD_SITUACAO_CANDIDATO_TOT as ID_SITUACAO_CANDIDATO_TOTALIZACAO,
        ST_PREST_CONTAS as REALIZOU_PRESTACAO_CONTAS,
        ST_SUBSTITUIDO as SUBSTITUIDO,
        SQ_SUBSTITUIDO as ID_CANDIDATO_SUBSTITUIDO,
        SQ_ORDEM_SUPLENCIA as ORDEM_SUPLENCIA,
        DT_ACEITE_CANDIDATURA as DATA_ACEITE_CANDIDATURA
    FROM
        infos_adicionais
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "informacoes_adicionais.parquet"), con)

con.close()


In [66]:
### EXTRAI TABELAS CONTIDAS EM candidatos.bens.parquet

import os

dataset_directory = os.path.join(os.getcwd(), "dataset", "2022")
tables_directory = os.path.join("dataset", "2022", "candidatos", "tabelas")
os.makedirs(tables_directory, exist_ok=True)

con = duckdb.connect()
candidatos_view = ParquetView(os.path.join(dataset_directory, "candidatos.bens.parquet"), "bens")
create_view([candidatos_view], con)


## TIPOS_BEM_CANDIDATO
query = """
   SELECT distinct
        CD_TIPO_BEM_CANDIDATO as ID,
        DS_TIPO_BEM_CANDIDATO as DESCRICAO
    FROM
        bens
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "tipos_bem.parquet"), con)

## BENS_CANDIDATO
query = """
   SELECT distinct
        SQ_CANDIDATO as ID_CANDIDATO,
        NR_ORDEM_BEM_CANDIDATO as ORDEM_BEM,
        CD_TIPO_BEM_CANDIDATO as ID_TIPO_BEM,
        DS_BEM_CANDIDATO as DESCRICAO,
        VR_BEM_CANDIDATO as VALOR,
        DT_ULT_ATUAL_BEM_CANDIDATO as DATA_ULTIMA_ATUALIZACAO,
        HH_ULT_ATUAL_BEM_CANDIDATO as HORA_ULTIMA_ATUALIZACAO
    FROM
        bens
"""
materialize_query_into_parquet(query, os.path.join(tables_directory, "bens.parquet"), con)


con.close()

In [10]:
import duckdb
import os

# Criar uma conexão DuckDB em memória
con = duckdb.connect()

# Carregar os dois arquivos Parquet
def create_views(con, candidatos_path, candidatos_bens_path):
    con.execute(f"CREATE VIEW candidatos AS SELECT * FROM '{candidatos_path}'")
    con.execute(f"CREATE VIEW candidatos_bens AS SELECT * FROM '{candidatos_bens_path}'")

candidatos_path = 'dataset/2022/candidatos.parquet'
candidatos_bens_path = 'dataset/2022/candidatos.bens.parquet'

create_views(con, candidatos_path, candidatos_bens_path)
con.execute("CREATE VIEW candidatos_bens AS SELECT * FROM 'dataset/2022/candidatos.bens.parquet'")


query = """
    SELECT 
        candidato.SQ_CANDIDATO,
        candidato.NM_CANDIDATO,
        candidato.NM_URNA_CANDIDATO,
        candidato.NR_CANDIDATO,
        candidato.CD_SITUACAO_CANDIDATURA,
        bem.*
    FROM candidatos cantidato
    inner join candidatos_bens bem on bem.SQ_CANDIDATO = cantidato.SQ_CANDIDATO
"""


# Executar a consulta para combinar as tabelas (JOIN, UNION, etc.)
query = """
    SELECT 
        candidatos.SQ_CANDIDATO,
        candidatos.NR_CANDIDATO,
        candidatos.NM_CANDIDATO,
        candidatos.NM_URNA_CANDIDATO,
        candidatos.CD_SITUACAO_CANDIDATURA,
        candidatos.NR_PARTIDO,
        candidatos.SG_PARTIDO,
        candidatos.SG_UF_NASCIMENTO,
        candidatos.DT_NASCIMENTO,
        candidatos.CD_GENERO,
        candidatos.DS_GENERO,
        candidatos.CD_GRAU_INSTRUCAO,
        candidatos.DS_GRAU_INSTRUCAO,
        candidatos.CD_ESTADO_CIVIL,
        candidatos.DS_ESTADO_CIVIL,
        candidatos.CD_COR_RACA,
        candidatos.DS_COR_RACA,
        candidatos.CD_OCUPACAO,
        candidatos.DS_OCUPACAO,
        candidatos.CD_SIT_TOT_TURNO,
        candidatos.DS_SIT_TOT_TURNO,
    FROM candidatos
"""
# Obter o resultado como um DataFrame
df_resultante = con.execute(query).fetchdf()

print(df_resultante)

views_directory = os.path.join(os.getcwd(), "dataset", "views")
os.makedirs(views_directory, exist_ok=True)

# Salvar o resultado em um novo arquivo Parquet
df_resultante.to_parquet(os.path.join(views_directory, "candidatos.dados.base.parquet"), index=False)

print("View materializada e salva com sucesso!")

       SQ_CANDIDATO  NR_CANDIDATO                      NM_CANDIDATO  \
0      100001608211          5120    BRUNNA DE SOUZA AMORIM MARCIEL   
1      250001619350         50110        EDIANE MARIA DO NASCIMENTO   
2       70001723597         20777    MARIA JOSÉ ALVES MORENO CABRAL   
3       80001719700         65077    CARMEM LUCIA GOMES DE OLIVEIRA   
4      190001619212          1388               ROBSON SOUZA SANTOS   
...             ...           ...                               ...   
29309  130001634154          1940              JOSE MARCIO DOS REIS   
29310  240001614291         14000          JULIANO DA SILVA MARTINS   
29311  240001644409         12212         ROSELÉIA LUCAS DOS SANTOS   
29312  140001596650            22    ROSIANE CHAGAS MESQUITA EGUCHI   
29313  170001736575         14300  GERCINALDO DO NASCIMENTO BARBOZA   

      NM_URNA_CANDIDATO  CD_SITUACAO_CANDIDATURA  NR_PARTIDO SG_PARTIDO  \
0        BRUNNA MARCIEL                       12          51   PATRIOTA 

In [ ]:
import duckdb
import pandas as pd

# Caminhos dos arquivos Parquet
arquivos_parquet = [
    'dataset/2022/candidatos.bens.parquet',
    'dataset/2022/candidatos.despesas.contratadas.parquet',
    'dataset/2022/candidatos.despesas.pagas.parquet',
    'dataset/2022/candidatos.infos_adicionais.parquet',
    'dataset/2022/candidatos.parquet',
    'dataset/2022/candidatos.receitas.doador_originario.parquet',
    'dataset/2022/candidatos.receitas.parquet',
    'dataset/2022/resultados.votacao.canditados.parquet'
]

# Conectar ao DuckDB e salvar o banco em um arquivo .db
con = duckdb.connect('meu_banco.db')

# Iterar sobre os arquivos Parquet e inserir os dados em uma tabela no banco .db
for arquivo in arquivos_parquet:
    # Ler o arquivo Parquet e inserir no banco DuckDB
    con.execute(f"COPY (SELECT * FROM '{arquivo}') TO 'meu_banco.db' (FORMAT PARQUET)")
    print(f"Arquivo {arquivo} inserido com sucesso no banco!")

# Verificar o conteúdo da tabela (opcional)
resultado = con.execute("SELECT * FROM 'meu_banco.db' LIMIT 10").fetchdf()
print(resultado)

# Fechar a conexão
con.close()

In [ ]:
import pyarrow.parquet as pq

# Ler o esquema do arquivo Parquet
arquivo_parquet = 'dataset/2022/candidatos.parquet'
parquet_file = pq.ParquetFile(arquivo_parquet)

# Exibir o esquema do arquivo Parquet
print(parquet_file.schema)

In [ ]:
import pandas as pd
import sqlite3
import os

# Diretório onde os arquivos .parquet estão localizados
diretorio_parquet = 'dataset/2022'
# Conecta ao banco de dados SQLite (ou cria um novo)
conn = sqlite3.connect('seu_banco_de_dados.sqlite')

# Itera sobre os arquivos .parquet no diretório
for arquivo in os.listdir(diretorio_parquet):
    if arquivo.endswith('.parquet'):
        # Lê o arquivo .parquet
        caminho_completo = os.path.join(diretorio_parquet, arquivo)
        df = pd.read_parquet(caminho_completo)
        
        # Define o nome da tabela com base no nome do arquivo
        nome_tabela = os.path.splitext(arquivo)[0]  # Remove a extensão .parquet

        # Insere os dados no banco de dados
        df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

# Fecha a conexão
conn.close()